### Welcome to v4 of Facebook Scraper-inator!
#### NO MORE CREDENTIALS. Just upload the brief.

####Import!

In [ ]:
!pip install selenium beautifulsoup4
!apt-get update
!apt install chromium-chromedriver
!pip install selenium-screenshot
!pip install webdriver-manager
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/

#### Install necessary libraries



In [ ]:
#data scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

#data handling
import urllib.request
import pandas as pd
import json
import time
import re
import datetime
import numpy as np
import openpyxl
import os
from IPython.display import FileLink
from tqdm.notebook import tqdm
import random
from IPython.display import HTML
from selenium.common.exceptions import TimeoutException

#### Reading the file

In [ ]:
target_directory = "/content/"
files = os.listdir(target_directory)
keyword = "brief"
file_format = ".xlsx"

matching_files = [f for f in files if keyword in f.lower() and f.endswith(file_format)]

if len(matching_files) == 0:
    print("No matching file found. Please upload an XLSX file with a similar name to 'brief'.")
else:
    try:
        matching_file = matching_files[0]
        file_path = os.path.join(target_directory, matching_file)
        df = pd.read_excel(file_path)
        print(f"Found a matching file: {matching_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Found a matching file: December FB brief.xlsx


#### Amending the file

In [ ]:
df.dropna(subset=[df.columns[1]], inplace=True)
df = df.reset_index(drop=True)
df['Post_Caption'] = None

In [ ]:
df.head()

,Movie Name,Post Name,Movie Type,Post_Caption
0,Kung Fu Panda,KFP4 - Trailer,Animation,None
1,NaN,KFP4 - poster,NaN,None
2,SMBM,Golden Globes - SMBM,NaN,None
3,Migration,Migration - Together Bonding,NaN,None
4,NaN,Migration - Premiere tickets giveaway,NaN,None


#### Extracting the embedded hyperlinks

In [ ]:
target_directory = "/content/"
keyword = "brief"
file_format = ".xlsx"

files = os.listdir(target_directory)

matching_files = [f for f in files if keyword in f.lower() and f.endswith(file_format)]

if not matching_files:
    print("No matching file found. Please upload an XLSX file with a similar name to 'brief'.")
else:
    matching_file = matching_files[0]
    try:
        file_path = os.path.join(target_directory, matching_file)
        wb = openpyxl.load_workbook(file_path)
        ws = wb['Sheet1']
        links = []


        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=2):
            cell = row[0]
            if cell.hyperlink is not None:
                links.append(cell.hyperlink.target)
            elif cell.value:
                links.append("No URL Available")
            else:
                links.append(None)

        links = [link for link in links if link is not None]

        df['Post URL'] = links
        print(f"Found a matching file: {matching_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


Found a matching file: December FB brief.xlsx


In [ ]:
df.head()

,Movie Name,Post Name,Movie Type,Post_Caption,Post URL
0,Kung Fu Panda,KFP4 - Trailer,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...
1,NaN,KFP4 - poster,NaN,None,https://www.facebook.com/UIPSingapore/posts/pf...
2,SMBM,Golden Globes - SMBM,NaN,None,https://www.facebook.com/UIPSingapore/posts/pf...
3,Migration,Migration - Together Bonding,NaN,None,https://www.facebook.com/UIPSingapore/posts/pf...
4,NaN,Migration - Premiere tickets giveaway,NaN,None,https://www.facebook.com/UIPSingapore/posts/pf...


#### Filling the null the values

In [ ]:
current_value_movie_name = None

for i, value in enumerate(df.iloc[:, 0]):
    if not pd.isna(value):
        current_value_movie_name = value
    else:
        df.iat[i, 0] = current_value_movie_name

current_value = None

for i, value in enumerate(df.iloc[:, 2]):
    if not pd.isna(value):
        current_value_movie_type = value
    else:
        df.iat[i, 2] = current_value_movie_type


In [ ]:
df.tail()

,Movie Name,Post Name,Movie Type,Post_Caption,Post URL
17,Bob Marley,Trailer launch,Drama,None,https://www.facebook.com/UIPSingapore/posts/pf...
18,Bob Marley,Poster launch,Drama,None,https://www.facebook.com/UIPSingapore/posts/pf...
19,IF,IF - Trailer,Drama,None,https://www.facebook.com/UIPSingapore/posts/pf...
20,IF,IF - Poster,Drama,None,https://www.facebook.com/UIPSingapore/posts/pf...
21,IF,IF - BTS Featurette,Drama,None,https://www.facebook.com/UIPSingapore/posts/pf...


In [ ]:
df.head()

,Movie Name,Post Name,Movie Type,Post_Caption,Post URL
0,Kung Fu Panda,KFP4 - Trailer,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...
1,Kung Fu Panda,KFP4 - poster,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...
2,SMBM,Golden Globes - SMBM,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...
3,Migration,Migration - Together Bonding,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...
4,Migration,Migration - Premiere tickets giveaway,Animation,None,https://www.facebook.com/UIPSingapore/posts/pf...


#### Intialize Chrome WebDriver

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-infobars")
browser = webdriver.Chrome(options=chrome_options)

#### Scraping Engine v4

In [ ]:
for index, row in df.iterrows():
    url = row['Post URL']
    sleep_duration = random.uniform(1, 2)
    browser.get(url)
    current_url = browser.current_url
    time.sleep(sleep_duration)

    if "videos" in current_url or "posts" in current_url or "events" in current_url:
        try:
            element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/head/meta[@name='description']"))
            )
            caption = element.get_attribute("content")
            df.at[index, 'Post_Caption'] = caption

        except TimeoutException:
            print(f"Timeout exception for URL: {url}")
            continue

    else:
        df.at[index, 'Post_Caption'] = None
        continue

df.head()

,Movie Name,Post Name,Movie Type,Post_Caption,Post URL
0,Kung Fu Panda,KFP4 - Trailer,Animation,Jack Black is back! Watch the trailer for #Kun...,https://www.facebook.com/UIPSingapore/posts/pf...
1,Kung Fu Panda,KFP4 - poster,Animation,Mastering the art of excitement! The Dragon Wa...,https://www.facebook.com/UIPSingapore/posts/pf...
2,SMBM,Golden Globes - SMBM,Animation,If you're wondering what's on the other end of...,https://www.facebook.com/UIPSingapore/posts/pf...
3,Migration,Migration - Together Bonding,Animation,"Well, whoever said family vacations were relax...",https://www.facebook.com/UIPSingapore/posts/pf...
4,Migration,Migration - Premiere tickets giveaway,Animation,[GIVEAWAY] Here's a special gift from the Mall...,https://www.facebook.com/UIPSingapore/posts/pf...


In [ ]:
df.head()

,Movie Name,Post Name,Movie Type,Post_Caption,Post URL
0,Kung Fu Panda,KFP4 - Trailer,Animation,Jack Black is back! Watch the trailer for #Kun...,https://www.facebook.com/UIPSingapore/posts/pf...
1,Kung Fu Panda,KFP4 - poster,Animation,Mastering the art of excitement! The Dragon Wa...,https://www.facebook.com/UIPSingapore/posts/pf...
2,SMBM,Golden Globes - SMBM,Animation,If you're wondering what's on the other end of...,https://www.facebook.com/UIPSingapore/posts/pf...
3,Migration,Migration - Together Bonding,Animation,"Well, whoever said family vacations were relax...",https://www.facebook.com/UIPSingapore/posts/pf...
4,Migration,Migration - Premiere tickets giveaway,Animation,[GIVEAWAY] Here's a special gift from the Mall...,https://www.facebook.com/UIPSingapore/posts/pf...


#### Trim and Clean

In [ ]:
def trim_and_clean(text):
    if isinstance(text, str):
        cleaned_text = text.strip()
        cleaned_text = cleaned_text.replace('\n', ' ')

        return cleaned_text
    else:
        return str(text)

df['Cleaned Post Caption'] = df['Post_Caption'].astype(str).apply(trim_and_clean)

####Downloading the File

In [ ]:
df2 = df.copy()
df2.drop(columns=['Post_Caption'], inplace=True)
df2.rename(columns={'Cleaned Post Caption': 'Post_Caption'}, inplace=True)

In [ ]:
xlsx_filename = 'FB Post Captions.xlsx'
df2.to_excel(xlsx_filename, index=False)

####Diagnostics/Termination

In [ ]:
#browser.save_screenshot("screenshot.png")

In [ ]:
browser.quit()

In [ ]:
print("Scraping Finished")

Scraping Finished


### -----------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# @title Progress!
HTML("<h1 style='font-size:24px; color:green;'>Run Complete! Refresh the left pane and download FB Post Captions.</h1>")